In [1]:
from jogador import Jogador
from baralho import Baralho
from tabela_de_pontos import TabelaDePontos
from constantes import manilhas, valores, naipes
import os
import platform

class Jogo():
    def __init__(self, num_jogadores):
        
        self.numero_jogadores = num_jogadores
        self.jogadores = self.obter_jogadores(num_jogadores)
        self.baralho = Baralho()
        self.tabela_pontos = TabelaDePontos()
        self.dict_pontos_cartas = self.obter_ordem_truco_mineiro()

    def _limpar_cmd():
        sistema_operacional = platform.system()

        if sistema_operacional == 'Windows':
            os.system('cls')
        else:
            os.system('clear')
    
    def obter_ordem_truco_mineiro(self):
        score = 1
        dict_pontos_cartas = {}
        for valor in valores:
            for naipe in naipes:
                carta = valor + naipe
                if carta not in manilhas: 
                    dict_pontos_cartas[carta] = score
            score += 1

        for manilha in manilhas:
            dict_pontos_cartas[manilha] = score
            score += 1
        return dict_pontos_cartas 
    
    def obter_jogadores(self, numero_jogadores):
        lista_de_jogadores = list()
        for id in range(1, numero_jogadores + 1):
            id_time = self.calcular_id_time_do_jogador(id)
            print("Escolha o nome do jogador "+str(id)+" (time "+str(id_time)+"): ")
            nome_do_jogador = input()
            lista_de_jogadores.append(Jogador(nome_do_jogador, id, id_time))
        return lista_de_jogadores
    
    def calcular_id_time_do_jogador(self , id_jogador):
        return str(2 - (id_jogador % 2))

In [2]:
jogo = Jogo(2)

Escolha o nome do jogador 1 (time 1): 
Escolha o nome do jogador 2 (time 2): 


In [3]:
jogo.jogadores

[ar2, ajsdiosa]